### Liens python :

- [docs.python](https://docs.python.org)
- [python.doctor](https://python.doctor)

### Projet :
- [CVE dataset](https://nvd.nist.gov/vuln/data-feeds)

- [CVE descriptif](https://www.redhat.com/fr/topics/security/what-is-cve)

- [CPE descriptif](https://medium.com/prohacktive/comment-exploiter-la-base-cve-du-nist-dfb10837da5c)

- [Standards pour la gestion des vulnérabilités](https://www.cert-ist.com/public/fr/SO_detail?code=standards_gestion_vulnerabilites)

### à voir :

- cartopy
- ipywidget
- [kaggle](https://www.kaggle.com/)

### notes :

***CWE*** est un système de catégorisation des types de vulnérabilité, tandis que ***CVE*** est une référence à une vulnérabilité spécifique. 

files.maxMemoryForLargeFilesMB


<hr>

<h2>Introduction</h2>

**Common Vulnerabilities and Exposures** ou **CVE** est un dictionnaire des informations publiques relatives aux vulnérabilités de sécurité. Le dictionnaire est maintenu par l'organisme MITRE, soutenu par le département de la Sécurité intérieure des États-Unis.

**Common Vulnerability Scoring System** ou **CVSS** est un système d'évaluation standardisé de la criticité des vulnérabilités selon des critères objectifs et mesurables.


CVSS se compose de trois groupes de mesures : Base, Temporel, et Environnemental. 
Les métriques de base produisent un score allant de 0 à 10, qui peut ensuite être modifié en notant les métriques temporelles et environnementales.
<br>
Un score CVSS est également représenté sous la forme d'une chaîne vectorielle, une représentation textuelle comprimée des valeurs utilisées pour obtenir le score. Ainsi, CVSS est bien adapté comme système de mesure standard pour les industries, les organisations et les gouvernements qui ont besoin de scores de gravité de vulnérabilité précis et cohérents.
<br>

La base de données nationale sur les vulnérabilités (**NVD**) fournit des scores CVSS pour presque toutes les vulnérabilités connues.
<br>

**La NVD prend en charge les normes CVSS v2.0 et v3.X.**

![cvss](media/cvss.png)

Le NVD fournit des "scores de base" CVSS qui représentent les caractéristiques innées de chaque vulnérabilité.
Le NVD ne fournit pas actuellement de "scores temporels" (mesures qui évoluent dans le temps en raison d'événements extérieurs à la vulnérabilité) ou de "scores environnementaux" (scores personnalisés pour refléter l'impact de la vulnérabilité sur votre organisation). Cependant, le NVD fournit un calculateur CVSS pour CVSS v2 et v3 qui vous permet d'ajouter des données de score temporel et environnemental.
<br>

Pour certaines vulnérabilités, toutes les informations nécessaires à la création des scores CVSS peuvent ne pas être disponibles. Cela se produit généralement lorsqu'un fournisseur annonce une vulnérabilité mais refuse de fournir certains détails. Dans de telles situations, les analystes du NVD attribuent des scores CVSS en utilisant l'approche du pire cas. Ainsi, si un fournisseur ne fournit aucun détail sur une vulnérabilité, le NVD attribuera à cette vulnérabilité une note de 10.0 (la note la plus élevée).



In [ ]:
# For Download all cve dataset from nvd.nist.gov
# import requests
# from datetime import datetime

# url = 'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{}.json.gz'

# for cve in range(2002, datetime.now().year+1):
#     data = requests.get(url.format(cve)).content
#     with open(f'cve_dataset/nvdcve-1.1-{cve}.json.gz','wb') as f:
#         f.write(data)

In [ ]:
import json
from glob import glob
import gzip
import pandas as pd
from pandas import json_normalize
import xml.etree.ElementTree as et

pd.options.display.max_columns = None

In [ ]:
# Extract gzip data and create CVE dataframe
cve_dataset = list()

for data in glob('cve_dataset/*2020.json.gz'):
    with open(data,'rb') as f:
        json_data = json.loads(gzip.decompress(f.read()))

        cve_items = json_normalize(json_data['CVE_Items'])
        cve_items.drop([
            'cve.data_type',
            'cve.data_format',
            'cve.data_version',
            'cve.references.reference_data',
            'configurations.CVE_data_version',
            'configurations.nodes',
            'cve.description.description_data'],
            axis=1,
            inplace=True
        )

        # maybe todo with map & list comprehension
        descriptions = json_normalize(
            json_data['CVE_Items'],
            record_path=[['cve','description','description_data']],
            meta=[['cve','CVE_data_meta','ID']]
        )
        descriptions.drop(['lang'],axis=1,inplace=True)


        dataframe = cve_items.merge(descriptions,on='cve.CVE_data_meta.ID')

        dataframe.rename(
            columns={
                "cve.CVE_data_meta.ID":"ID",
                "cve.problemtype.problemtype_data":"cwe_ids",
                "value":"description"
            },
            inplace=True
        )

        dataframe = dataframe.set_index("ID")
        cve_dataset.append(dataframe)

CVE_df = pd.concat(cve_dataset)

CVE_df.insert(2, "cwe_names", None)

CVE_df= CVE_df[sorted(CVE_df)]

In [ ]:
# extract CWE ids
CVE_df['cwe_ids'] = \
CVE_df['cwe_ids'].map(lambda x: [id['value'] for sublist in x for id in sublist['description']])


In [ ]:
# Use Common Weakness Enumeration (CWE) for CVE type categorisation
cwe_dataset = dict()
tree = et.parse('cve_dataset/cwec_v4.7.xml')
root = tree.getroot()

for weakness in root.iter('{http://cwe.mitre.org/cwe-6}Weakness'):
    cwe_dataset[weakness.get('ID')] = weakness.get('Name')

CWE_df = pd.DataFrame.from_dict(cwe_dataset, orient='index', columns=["CWE_NAME"])

In [ ]:
# mapping cwe_id with cwe_name

def categorise(row):    
    cwe_names = []
    for id in row['cwe_ids']:
        try:
            cwe_names.append(CWE_df.loc[id.split("-",1)[1]]['CWE_NAME'])
        except:
            cwe_names.append(id)

    return cwe_names


CVE_df['cwe_names'] = CVE_df.apply(lambda row: categorise(row), axis=1)

In [20]:
CVE_df.loc['CVE-2020-1695']

CVE_df.sample(20)

,cve.CVE_data_meta.ASSIGNER,cwe_ids,cwe_names,description,impact.baseMetricV2.acInsufInfo,impact.baseMetricV2.cvssV2.accessComplexity,impact.baseMetricV2.cvssV2.accessVector,impact.baseMetricV2.cvssV2.authentication,impact.baseMetricV2.cvssV2.availabilityImpact,impact.baseMetricV2.cvssV2.baseScore,impact.baseMetricV2.cvssV2.confidentialityImpact,impact.baseMetricV2.cvssV2.integrityImpact,impact.baseMetricV2.cvssV2.vectorString,impact.baseMetricV2.cvssV2.version,impact.baseMetricV2.exploitabilityScore,impact.baseMetricV2.impactScore,impact.baseMetricV2.obtainAllPrivilege,impact.baseMetricV2.obtainOtherPrivilege,impact.baseMetricV2.obtainUserPrivilege,impact.baseMetricV2.severity,impact.baseMetricV2.userInteractionRequired,impact.baseMetricV3.cvssV3.attackComplexity,impact.baseMetricV3.cvssV3.attackVector,impact.baseMetricV3.cvssV3.availabilityImpact,impact.baseMetricV3.cvssV3.baseScore,impact.baseMetricV3.cvssV3.baseSeverity,impact.baseMetricV3.cvssV3.confidentialityImpact,impact.baseMetricV3.cvssV3.integrityImpact,impact.baseMetricV3.cvssV3.privilegesRequired,impact.baseMetricV3.cvssV3.scope,impact.baseMetricV3.cvssV3.userInteraction,impact.baseMetricV3.cvssV3.vectorString,impact.baseMetricV3.cvssV3.version,impact.baseMetricV3.exploitabilityScore,impact.baseMetricV3.impactScore,lastModifiedDate,publishedDate
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CVE-2020-1081,secure@microsoft.com,[CWE-20],[Improper Input Validation],An elevation of privilege vulnerability exists...,False,LOW,LOCAL,NONE,COMPLETE,7.2,COMPLETE,COMPLETE,AV:L/AC:L/Au:N/C:C/I:C/A:C,2.0,3.9,10.0,False,False,False,HIGH,False,LOW,LOCAL,HIGH,7.8,HIGH,HIGH,HIGH,LOW,UNCHANGED,NONE,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,3.1,1.8,5.9,2021-07-21 11:39:00+00:00,2020-05-21 23:15:00+00:00
CVE-2020-1490,secure@microsoft.com,[NVD-CWE-noinfo],[NVD-CWE-noinfo],An elevation of privilege vulnerability exists...,False,LOW,LOCAL,NONE,PARTIAL,4.6,PARTIAL,PARTIAL,AV:L/AC:L/Au:N/C:P/I:P/A:P,2.0,3.9,6.4,False,False,False,MEDIUM,False,LOW,LOCAL,HIGH,7.8,HIGH,HIGH,HIGH,LOW,UNCHANGED,NONE,CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,3.1,1.8,5.9,2021-07-21 11:39:00+00:00,2020-08-17 19:15:00+00:00
CVE-2020-9875,product-security@apple.com,[CWE-190],[Integer Overflow or Wraparound],An integer overflow was addressed through impr...,False,MEDIUM,NETWORK,NONE,PARTIAL,6.8,PARTIAL,PARTIAL,AV:N/AC:M/Au:N/C:P/I:P/A:P,2.0,8.6,6.4,False,False,False,MEDIUM,True,LOW,LOCAL,HIGH,7.8,HIGH,HIGH,HIGH,NONE,UNCHANGED,REQUIRED,CVSS:3.1/AV:L/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H,3.1,1.8,5.9,2020-10-27 19:58:00+00:00,2020-10-22 18:15:00+00:00
CVE-2020-24303,cve@mitre.org,[CWE-79],[Improper Neutralization of Input During Web P...,Grafana before 7.1.0-beta 1 allows XSS via a q...,False,MEDIUM,NETWORK,NONE,NONE,4.3,NONE,PARTIAL,AV:N/AC:M/Au:N/C:N/I:P/A:N,2.0,8.6,2.9,False,False,False,MEDIUM,True,LOW,NETWORK,NONE,6.1,MEDIUM,LOW,LOW,NONE,CHANGED,REQUIRED,CVSS:3.1/AV:N/AC:L/PR:N/UI:R/S:C/C:L/I:L/A:N,3.1,2.8,2.7,2020-11-23 12:15:00+00:00,2020-10-28 14:15:00+00:00
CVE-2020-27048,security@android.com,[CWE-787],[Out-of-bounds Write],"In RW_SendRawFrame of rw_main.cc, there is a p...",False,MEDIUM,NETWORK,NONE,PARTIAL,6.8,PARTIAL,PARTIAL,AV:N/AC:M/Au:N/C:P/I:P/A:P,2.0,8.6,6.4,False,False,False,MEDIUM,True,LOW,LOCAL,HIGH,7.8,HIGH,HIGH,HIGH,NONE,UNCHANGED,REQUIRED,CVSS:3.1/AV:L/AC:L/PR:N/UI:R/S:U/C:H/I:H/A:H,3.1,1.8,5.9,2020-12-16 18:18:00+00:00,2020-12-15 17:15:00+00:00
CVE-2020-10239,cve@mitre.org,[CWE-863],[Incorrect Authorization],An issue was discovered in Joomla! before 3.9....,False,LOW,NETWORK,SINGLE,PARTIAL,6.5,PARTIAL,PARTIAL,AV:N/AC:L/Au:S/C:P/I:P/A:P,2.0,8.0,6.4,False,False,False,MEDIUM,False,LOW,NETWORK,HIGH,8.8,HIGH,HIGH,HIGH,LOW,UNCHANGED,NONE,CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H,3.1,2.8,5.9,2021-07-21 11:39:00+00:00,2020-03-16 16:15:00+00:00
CVE-2020-4230,psirt@us.ibm.com,[NVD-CWE-noinfo],[NVD-CWE-noinfo],"IBM DB2 for Linux, UNIX and Windows (includes ...",False,LOW,LOCAL,NONE,PARTIAL,4.6,PARTIAL,PARTIAL,A

In [ ]:
## cast date
CVE_df['publishedDate'] = pd.to_datetime(CVE_df['publishedDate'])
CVE_df['lastModifiedDate'] = pd.to_datetime(CVE_df['lastModifiedDate'])

In [ ]:
# displays statistics for quantitative variables
CVE_df.describe()

In [ ]:
# find by CVE-id
CVE_df.loc['CVE-2005-1479']

In [ ]:
CVE_df.isnull().sum()

In [ ]:
CVE_df.duplicated().sum()

# ** REJECT ** DO NOT USE THIS CANDIDATE NUMBER
mask = CVE_df.duplicated()

CVE_df[mask]

In [ ]:
# test graph
years = CVE_df['publishedDate'].dt.year

years[years == 1989]
# a = years.value_counts().sort_index()

# years.value_counts().sort_index().plot(kind='bar',figsize=(15, 5),logy=True)
